<a href="https://colab.research.google.com/github/neerajguleria1/driver-drowsiness-detection/blob/main/Untitled24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile system_pipeline.py
import joblib
import numpy as np
from typing import Dict, List


class DriverSafetySystem:
    """
    Decision Intelligence System.
    ML is ONLY one component.
    """

    def __init__(self, model_path: str):
        try:
            self.model = joblib.load(model_path)
        except Exception as e:
            raise RuntimeError(f"Failed to load model: {e}")

    def health_check(self):
        if self.model is None:
            raise RuntimeError("Model not loaded")

    def analyze(self, input_data: Dict) -> Dict:
        features = self._prepare_features(input_data)

        # ML inference
        prob = self.model.predict_proba([features])[0]
        pred_index = int(np.argmax(prob))

        ml_prediction = "Drowsy" if pred_index == 1 else "Alert"
        ml_confidence = float(prob[pred_index])

        # Safety bounds
        ml_confidence = min(max(ml_confidence, 0.0), 1.0)

        # Decision Intelligence
        risk_score = self._compute_risk_score(input_data, ml_confidence)
        risk_state = self._map_risk_state(risk_score)
        decision = self._decision_engine(risk_state)
        explanations = self._generate_explanations(input_data, ml_prediction)

        return {
            "ml_prediction": ml_prediction,
            "ml_confidence": ml_confidence,
            "risk_score": risk_score,
            "risk_state": risk_state,
            "decision": decision,
            "explanations": explanations
        }

    def _prepare_features(self, data: Dict) -> List[float]:
        return [
            data["Speed"],
            data["Alertness"],
            data["Seatbelt"],
            data["Heart_rate"],
            data["Fatigue"],
            data["speed_change"],
            data["prev_alertness"]
        ]

    def _compute_risk_score(self, data: Dict, confidence: float) -> int:
        score = 0

        if data["Fatigue"] > 6:
            score += 30
        if data["Alertness"] < 0.5:
            score += 25
        if data["Heart_rate"] > 100:
            score += 15
        if confidence > 0.7:
            score += 20

        return min(max(score, 0), 100)

    def _map_risk_state(self, score: int) -> str:
        if score >= 70:
            return "CRITICAL"
        elif score >= 40:
            return "MODERATE"
        return "LOW"

    def _decision_engine(self, state: str) -> Dict:
        if state == "CRITICAL":
            return {
                "action": "Recommend immediate break",
                "severity": "HIGH",
                "message": "Risk increasing rapidly"
            }
        elif state == "MODERATE":
            return {
                "action": "Suggest rest soon",
                "severity": "MEDIUM",
                "message": "Driver fatigue detected"
            }
        else:
            return {
                "action": "No action required",
                "severity": "LOW",
                "message": "Driver condition normal"
            }

    def _generate_explanations(self, data: Dict, prediction: str) -> List[str]:
        reasons = []

        if data["Fatigue"] > 6:
            reasons.append("Fatigue level is high")
        if data["Alertness"] < 0.5:
            reasons.append("Alertness level is low")
        if data["Heart_rate"] > 100:
            reasons.append("Heart rate indicates stress")
        if prediction == "Drowsy":
            reasons.append("ML model detected drowsiness pattern")

        return reasons


Writing system_pipeline.py


In [7]:
import os
os.listdir()


['.config',
 'final_driver_drowsiness_pipeline.pkl',
 'system_pipeline',
 'sample_data']